# Modern Deck Prices Cleaning 

In [1]:
import pandas as pd
import requests
import json

Get all the cards from Scryfall, filter them, and clean them for compatibility with decklist formatting

In [2]:
#api call and cleaning
bulk = requests.get('https://api.scryfall.com/bulk-data') #I learned that the uri for bulk data changes daily, so requesting the bulk data for the day then getting the uri for updated cards is needed (I had outdated data)
bulk_df = pd.json_normalize(bulk.json(),'data')
oracle_uri = bulk_df['download_uri'].loc[bulk_df.query('type == "oracle_cards"').index[0]]

response = requests.get(oracle_uri)
raw_oracle_cards = pd.json_normalize(response.json())

oracle_cards = raw_oracle_cards[[#'object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id',
       #'mtgo_foil_id', 'tcgplayer_id', 'cardmarket_id', 
       'name', 
       #'lang',
       #'released_at', 'uri', 'scryfall_uri', 
       'layout', 
       #'highres_image',
       #'image_status', 'mana_cost', 'cmc', 'type_line', 'oracle_text',
       #'colors', 'color_identity', 'keywords', 'games', 'reserved',
       #'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint',
       #'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri',
       #'set_search_uri', 'scryfall_set_uri', 'rulings_uri',
       #'prints_search_uri', 'collector_number', 'digital', 
       #'rarity',
       #'flavor_text', 'card_back_id', 'artist', 'artist_ids',
       #'illustration_id', 'border_color', 'frame', 'full_art', 'textless',
       #'booster', 'story_spotlight', 'edhrec_rank', 'image_uris.small',
       #'image_uris.normal', 'image_uris.large', 'image_uris.png',
       #'image_uris.art_crop', 'image_uris.border_crop',
       #'legalities.standard',
       #'legalities.future', 'legalities.historic',
       #'legalities.gladiator', 
       #'legalities.pioneer',
       #'legalities.explorer', 
       'legalities.modern', 
       #'legalities.legacy',
       #'legalities.pauper', 'legalities.vintage', 'legalities.penny',
       #'legalities.commander', 'legalities.brawl',
       #'legalities.historicbrawl', 'legalities.alchemy',
       #'legalities.paupercommander', 'legalities.duel',
       #'legalities.oldschool', 'legalities.premodern', 
       'prices.usd',
       #'prices.usd_foil', 'prices.usd_etched', 'prices.eur',
       #'prices.eur_foil', 
       'prices.tix', 
       #'related_uris.gatherer',
       #'related_uris.tcgplayer_infinite_articles',
       #'related_uris.tcgplayer_infinite_decks', 'related_uris.edhrec',
       #'security_stamp', 'preview.source', 'preview.source_uri',
       #'preview.previewed_at', 'power', 'toughness', 'penny_rank',
       #'arena_id', 'watermark', 'produced_mana', 'all_parts',
       'card_faces', 
       #'frame_effects', 'tcgplayer_etched_id',
       #'promo_types', 'loyalty', 'life_modifier', 'hand_modifier',
       #'attraction_lights', 'color_indicator', 'content_warning'
       ]]
oracle_cards = oracle_cards.rename(columns = {
    'name':'Name',
    'rarity':'Rarity',
    'legalities.modern':'Modern_Legal',
    'prices.usd':'Price_USD',
    'prices.tix':'Price_Tix'
})
modern_raw = oracle_cards.query("Modern_Legal == 'legal'").sort_values(by = ['Name']).reset_index(drop = True)

#fix multi-faced cards to match mtggoldfish naming conventions (dual faced cards, split cards)
card_faces = pd.json_normalize(modern_raw['card_faces'].loc[~modern_raw['card_faces'].isna()])
front_face = pd.json_normalize(card_faces[0])
back_face = pd.json_normalize(card_faces[1])

faces_index = 0 # we lose the relationship between the indices when we normalize the faces data, I'm not sure how to fix this with pandas but just keeping track of it like this works fine
for card_index in modern_raw.loc[~modern_raw['card_faces'].isna()].index:
    if modern_raw['layout'].loc[card_index] == 'split':
        modern_raw['Name'].loc[card_index] = front_face['name'].loc[faces_index]+'/'+back_face['name'].loc[faces_index]
    else:
        modern_raw["Name"].loc[card_index] = front_face['name'].loc[faces_index]
    faces_index += 1
modern = modern_raw.drop(['card_faces','layout'],axis=1)
modern.head(10)

,Name,Modern_Legal,Price_USD,Price_Tix
0,+2 Mace,legal,0.02,0.02
1,A Little Chat,legal,0.04,0.01
2,Abandon Reason,legal,0.12,0.03
3,Abandon the Post,legal,0.01,0.03
4,Abandoned Sarcophagus,legal,0.10,None
5,Abattoir Ghoul,legal,0.13,None
6,Abbey Griffin,legal,0.05,0.03
7,Abbot of Keral Keep,legal,0.20,0.02
8,Aberrant Mind Sorcerer,legal,0.04,0.02
9,Aberrant Researcher,legal,0.06,0.03


Read in all the decklists and put them into a dataframe for the decks

In [3]:
decks = pd.DataFrame(columns=['Deck_Name','Deck_List','Paper_Price','MTGO_Price_Tix'])
modern_deck_names = ['Izzet Murktide','5 Color Indomitable Creativity','Hammer Time','Rakdos Midrange','Burn','4 Color Omnath',
         'Amulet Titan','Golgari Yawgmoth','Domain Zoo','Mono-Green Tron','Dimir Mill','Generic Ragavan','Living End',
         'Merfolk',"Death's Shadow",'Jeskai Control','5 Color Reanimator','Mono-Blue Affinity','Jeskai Prowess','Azorius Control']

decks['Deck_Name'] = modern_deck_names

decklists = []
for i in decks.index:
    decklists.append(pd.read_csv('Deck Lists/Modern Metagame Decks/'+decks['Deck_Name'].loc[i]+'.csv'))
decks['Deck_List'] = decklists

decks[['Deck_Name','Deck_List']].head(len(decks))

,Deck_Name,Deck_List
0,Izzet Murktide,Quantity Card_Name 0 ...
1,5 Color Indomitable Creativity,Quantity Card_Name 0 ...
2,Hammer Time,Quantity Card_Name 0 ...
3,Rakdos Midrange,Quantity Card_Name 0...
4,Burn,Quantity Card_Name 0...
5,4 Color Omnath,Quantity Card_Name 0 ...
6,Amulet Titan,Quantity Card_Name ...
7,Golgari Yawgmoth,Quantity Card_Name 0 ...
8,Domain Zoo,Quantity Card_Name 0 ...
9,Mono-Green Tron,Quantity Card_Name 0 ...


Calculate prices and output the final data

In [4]:
def paper_price(decklist):
    deck_price = 0
    #with the decklist, we want to find the price from out standard dataframe, then multiply it by the quantity of card, then add that to price
    for card_index in range(len(decklist.index)): #iterate through the cards
        #get the individual card names and quantities
        card_name = decklist['Card_Name'].loc[card_index]
        card_quantity = decklist['Quantity'].loc[card_index]

        #find the card in the standard cardlist dataframe
        price_index = modern.query('Name == "'+card_name+'"')['Price_USD'].index[0]
        if modern['Price_USD'].loc[price_index]:
            card_price = float(modern['Price_USD'].loc[price_index])
        else:
            card_price = 0

        #add the price of the card for each time that the card 
        deck_price += (card_price * card_quantity)
        

    return deck_price

def mtgo_price(decklist):
    deck_price = 0
    #with the decklist, we want to find the price from out standard dataframe, then multiply it by the quantity of card, then add that to price
    for card_index in range(len(decklist.index)): #iterate through the cards
        #get the individual card names and quantities
        card_name = decklist['Card_Name'].loc[card_index]
        card_quantity = decklist['Quantity'].loc[card_index]

        #find the card in the standard cardlist dataframe
        price_index = modern.query('Name == "'+card_name+'"')['Price_USD'].index[0]
        if modern['Price_Tix'].loc[price_index]:
            card_price = float(modern['Price_Tix'].loc[price_index])
        else:
            card_price = 0

        #add the price of the card for each time that the card 
        deck_price += (card_price * card_quantity)
        

    return deck_price

In [5]:
for i in decks.index:
    decks['Paper_Price'].loc[i] = paper_price(decks['Deck_List'].loc[i])
    decks['MTGO_Price_Tix'].loc[i] = mtgo_price(decks['Deck_List'].loc[i])

decks[['Deck_Name','Paper_Price','MTGO_Price_Tix']].to_csv('Format Price Data/Modern_Metagame_Deck_Prices.csv',index=False)